### A. Set up HMM model

In [1]:
import time

states = ('Rainy', 'Sunny')
 
observations = ('walk', 'shop', 'clean')
 
initial_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_matrix = {
   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
   }
 
emission_matrix = {
   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
   }

# modified from wiki: https://en.wikipedia.org/wiki/Hidden_Markov_model

### B. Input a sequence of obervations vector

In [2]:
sequence_observations = ('shop', 'walk', 'clean', 'shop', 'walk')

### C. Probablity of the sequence observation 

##### C.1 Forward Algorithm
Let $z_k$ be the $k$th hidden state that can take $m$ number of discrete values, e.g. ('Rainy', 'Sunny'), and $x_{1:k}=(x_1, x_2, ... , x_k)$ is the observation vector, where each $x_i$ is one of for example ('walk', 'shop', 'clean').

Compute $P(z_k, x_{1:k})$ first,

$\begin{align*}
P(z_k, x_{1:k})&=\sum_{z_{k-1=1}}^{m}P(z_k. z_{k-1}, x_{1:k})\\
&=\sum_{z_{k-1}=1}P(x_k|z_k, z_{k-1}, x_{1:k-1})P(z_k|z_{k-1},x_{1:k-1})P(z_{k-1}|x_{1:k-1})P(x_{1:k-1})\\
\end{align*}$

from Markov property,

$\begin{equation*}
P(z_k, x_{1:k})=\sum_{z_{k-1}=1}P(x_k|z_k)P(z_k|z_{k-1})P(z_{k-1},x_{1:k-1})
\end{equation*}$

notice how $P(z_k, x_{1:k})$ reoccurs, we can therefore apply recursion or dynamic programming.

In [3]:
def forward_recur(z, x, tm=transition_matrix, em=emission_matrix, init=initial_probability):
    ''' z(str): state, e.g. rainny or sunny.
        x(tuple of strs): sequence of observation.
        tm(dict): transition matrix.
        em(dict): emission matrix.
        init(dict): initial probablity.'''
    m = tm.keys() # states
    if len(x) == 1:
        return em[z][x[0]]*init[z]
    pzx = 0.
    for i in range(len(m)):
        pzx += em[z][x[-1]]*tm[m[i]][z]*forward_recur(m[i], x[:-1])
    return pzx

# sum up joint dist
t_start = time.time()
p_joint = [forward_recur(i, sequence_observations) for i in transition_matrix.keys()]
t_end = time.time()
print "Time spent: {}s".format(t_end-t_start)
print "Probability of the observations vector is: ", sum(p_joint)

Time spent: 0.000108957290649s
Probability of the observations vector is:  0.003286224


##### C.2 Exhaustive Search
Given a sequence of observed vectors, we compute the sum of probabilities of all the possible configuration of states.

In [4]:
from itertools import product

def exhaustive(x, tm=transition_matrix, em=emission_matrix, init=initial_probability):
    m = len(tm.keys()) # number of state
    n = len(x) # length of observation
    result = 0.
    for comb in product(tm.keys(), repeat=n):
        p_comb = init[comb[0]]*em[comb[0]][x[0]]
        for i in range(1,n): # iterate through rest of sequence
            p_comb *= tm[comb[i-1]][comb[i]]*em[comb[i]][x[i]]
        result += p_comb
    return result

t_start = time.time()  
p = exhaustive(sequence_observations)
t_end = time.time()
print "Time spent: {}s".format(t_end-t_start)
print "Probability of the observations vector is: ", p

Time spent: 8.29696655273e-05s
Probability of the observations vector is:  0.003286224


Compare *C.1* and *C.2* we see there result match.

(notice how exhaustive search takes shorter time than recursion method in this case, but it would take much more time than recursion when length of sequence of observations gets large)